<a href="https://colab.research.google.com/github/A-P-Dharanya/Zeotap/blob/main/DHARANYA_AP_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Task 1: Exploratory Data Analysis (EDA)

import pandas as pd
import matplotlib.pyplot as plt

# load datasets
customers_file = '/Customers.csv'
products_file = '/Products.csv'
transactions_file = '/Transactions.csv'

customers_df = pd.read_csv(customers_file)
products_df = pd.read_csv(products_file)
transactions_df = pd.read_csv(transactions_file)

missing_customers = customers_df.isnull().sum()
missing_products = products_df.isnull().sum()
missing_transactions = transactions_df.isnull().sum()
duplicates_customers = customers_df.duplicated().sum()
duplicates_products = products_df.duplicated().sum()
duplicates_transactions = transactions_df.duplicated().sum()

# Converting date columns to datetime
customers_df['SignupDate'] = pd.to_datetime(customers_df['SignupDate'], errors='coerce')
transactions_df['TransactionDate'] = pd.to_datetime(transactions_df['TransactionDate'], errors='coerce')
(missing_customers, missing_products, missing_transactions,
 duplicates_customers, duplicates_products, duplicates_transactions)

#1. Data Cleaning

print("Missing values in Customers dataset:\n", customers_df.isnull().sum())
print("Missing values in Products dataset:\n", products_df.isnull().sum())
print("Missing values in Transactions dataset:\n", transactions_df.isnull().sum())

print("Duplicates in Customers dataset:", customers_df.duplicated().sum())
print("Duplicates in Products dataset:", products_df.duplicated().sum())
print("Duplicates in Transactions dataset:", transactions_df.duplicated().sum())

# Convert date columns to datetime
customers_df['SignupDate'] = pd.to_datetime(customers_df['SignupDate'])
transactions_df['TransactionDate'] = pd.to_datetime(transactions_df['TransactionDate'])

#2. Merging Datasets

merged_data = pd.merge(transactions_df, customers_df, on='CustomerID', how='inner')
merged_data = pd.merge(merged_data, products_df, on='ProductID', how='inner')
print("Merged dataset:\n", merged_data.head())

#3. Exploratory Data Analysis

region_sales = merged_data.groupby('Region')['TotalValue'].sum().sort_values(ascending=False)

# Visualization
region_sales.plot(kind='bar', title='Total Sales by Region', ylabel='Sales ($)', xlabel='Region')
plt.show()

#Most Purchased Products
product_popularity = merged_data['ProductName'].value_counts().head(10)

# Visualization
product_popularity.plot(kind='bar', title='Top 10 Most Purchased Products', ylabel='Count', xlabel='Product Name')
plt.show()

#Monthly Revenue Trend
merged_data['TransactionMonth'] = merged_data['TransactionDate'].dt.to_period('M')
monthly_revenue = merged_data.groupby('TransactionMonth')['TotalValue'].sum()

# Visualization
monthly_revenue.plot(title='Monthly Revenue Trend', ylabel='Revenue ($)', xlabel='Month')
plt.show()
customer_clv = merged_data.groupby('CustomerID')['TotalValue'].sum().sort_values(ascending=False)

print("Top 5 Customers by CLV:\n", customer_clv.head())

#Additional EDA

# 1. Customer Segmentation by Total Spend

customer_spend = merged_data.groupby('CustomerID')['TotalValue'].sum()

segments = ['Low Value', 'Medium Value', 'High Value']
customer_spend_category = pd.cut(customer_spend, bins=[0, 500, 1000, float('inf')], labels=segments)

# Add the segments to the merged data
customer_spend = customer_spend.to_frame()

customer_spend['SpendCategory'] = customer_spend_category

# Result
print("Customer Segmentation:\n", customer_spend.head())
print("Customer Segmentation:\n", customer_spend.head())

# 2. Average Transaction Value

# Calculate average transaction value
average_transaction_value = merged_data['TotalValue'].mean()
print(f"Average Transaction Value: ${average_transaction_value:.2f}")

# Average transaction value by region
avg_transaction_by_region = merged_data.groupby('Region')['TotalValue'].mean()

# Visualization
avg_transaction_by_region.plot(kind='bar', title='Average Transaction Value by Region', ylabel='Average Transaction Value ($)', xlabel='Region')
plt.show()

# 3. Product-wise Revenue

# Calculate total revenue by product
product_revenue = merged_data.groupby('ProductName')['TotalValue'].sum().sort_values(ascending=False)

# Visualization
product_revenue.head(10).plot(kind='bar', title='Top 10 Products by Revenue', ylabel='Revenue ($)', xlabel='Product Name')
plt.show()

# 4. Time-based Analysis of Sales

merged_data['DayOfWeek'] = merged_data['TransactionDate'].dt.day_name()
merged_data['HourOfDay'] = merged_data['TransactionDate'].dt.hour

sales_by_day = merged_data.groupby('DayOfWeek')['TotalValue'].sum().reindex(
    ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])

sales_by_day.plot(kind='bar', title='Total Sales by Day of Week', ylabel='Sales ($)', xlabel='Day of Week')
plt.show()
sales_by_hour = merged_data.groupby('HourOfDay')['TotalValue'].sum()

sales_by_hour.plot(kind='line', title='Total Sales by Hour of Day', ylabel='Sales ($)', xlabel='Hour of Day')
plt.show()

signup_trends = customers_df.groupby(customers_df['SignupDate'].dt.to_period('M'))['CustomerID'].count()

# Visualization
signup_trends.plot(title='Customer Signup Trends', ylabel='New Customers', xlabel='Month')
plt.show()

customer_clv_with_signup = merged_data.groupby('CustomerID').agg({
    'TotalValue': 'sum',
    'SignupDate': 'min'
}).reset_index()
customer_clv_with_signup['CustomerLifetime'] = (pd.to_datetime('today') - customer_clv_with_signup['SignupDate']).dt.days

# Visualize CLV vs Customer Lifetime
plt.scatter(customer_clv_with_signup['CustomerLifetime'], customer_clv_with_signup['TotalValue'])
plt.title('CLV vs Customer Lifetime')
plt.xlabel('Customer Lifetime (Days)')
plt.ylabel('Customer Lifetime Value ($)')
plt.show()

# Add the segments to the merged data

customer_spend = merged_data.groupby('CustomerID')['TotalValue'].sum()
segments = ['Low Value', 'Medium Value', 'High Value']
customer_spend_category = pd.cut(customer_spend, bins=[0, 500, 1000, float('inf')], labels=segments)
customer_spend_with_category = customer_spend.to_frame()
customer_spend_with_category['SpendCategory'] = customer_spend_category
print("Customer Segmentation:\n", customer_spend_with_category.head())
customer_spend_category = pd.cut(customer_spend, bins=[0, 500, 1000, float('inf')], labels=segments)
customer_spend_with_category = customer_spend.to_frame().copy()
customer_spend_with_category['SpendCategory'] = customer_spend_category

print("Customer Segmentation:\n", customer_spend_with_category.head())


